In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

@inproceedings{hateoffensive, title = {Automated Hate Speech Detection and the Problem of Offensive Language}, author = {Davidson, Thomas and Warmsley, Dana and Macy, Michael and Weber, Ingmar}, booktitle = {Proceedings of the 11th International AAAI Conference on Web and Social Media}, series = {ICWSM '17}, year = {2017}, location = {Montreal, Canada}, pages = {512-515} }
the second dataset has an MIT license

A basic test for a scoring algoritm based on the above dataset

Why? it is a rich dataset for offensive and hate speech tagging. 
If the test is ok, the model can be applied directly without the initial toxic dataset. 

In [ ]:
import csv
import requests
import gc

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import rankdata

In [ ]:
#test="test"
sample=0.01
test="final"

In [ ]:
jigsaw_df = pd.read_csv('../input/jigsaw-toxic-coomens-annotations/jigsaw_train_hate_annotationprob.csv')
#jigsaw_df=jigsaw_df.sample(300)
if test=="test":
    jigsaw_df=jigsaw_df.sample(frac=sample)

In [ ]:
sns.set_theme(style="darkgrid")

In [ ]:
jigsaw_df.head()

Interannotators agreement as neutral.

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"])

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"]**(jigsaw_df["neutral_speech_agreement"]+0.00001))

In [ ]:
gc.collect

In [ ]:
sns.distplot(jigsaw_df["offensive_agreement_rating"])

In [ ]:
gc.collect

In [ ]:

sns.distplot(jigsaw_df["hate_speech_agreement"])

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"]**jigsaw_df["neutral_speech_agreement"]+jigsaw_df["severity_filter"]*jigsaw_df["offensive_agreement_rating"]+2*jigsaw_df["severity_filter"]*jigsaw_df["hate_speech_agreement"])

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"]**jigsaw_df["neutral_speech_agreement"]+jigsaw_df["severity_filter"]*jigsaw_df["offensive_agreement_rating"])

Cutoff

In [ ]:
def cutoff (row):
    if row <0.5:        
        return 0
    else:
        return row

In [ ]:
jigsaw_df['cutoff_hate']=jigsaw_df['hate_speech_agreement'].apply(lambda x: cutoff(x))
jigsaw_df['cutoff_offensive']=jigsaw_df['offensive_agreement_rating'].apply(lambda x: cutoff(x))

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"]**jigsaw_df["neutral_speech_agreement"]+jigsaw_df['cutoff_offensive'])

In [ ]:
sns.distplot(jigsaw_df["neutral_speech_agreement"]**jigsaw_df["neutral_speech_agreement"]+jigsaw_df['cutoff_offensive']+jigsaw_df['cutoff_hate'])

In [ ]:
sns.distplot((1-jigsaw_df["neutral_speech_agreement"])+jigsaw_df['cutoff_offensive']+jigsaw_df['cutoff_hate'])

In [ ]:
sns.distplot(1-jigsaw_df["neutral_speech_agreement"])

In [ ]:
sns.distplot(1-jigsaw_df["neutral_speech_agreement"]+2*jigsaw_df['cutoff_offensive']+3*jigsaw_df['cutoff_hate'])

Load data for toxic-severity rating

In [ ]:
comments_to_score=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv",index_col=0)
if test =='test':
    comments_to_score=comments_to_score.sample(frac=sample)

In [ ]:
df_test=pd.read_csv("/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv",index_col=0)
if test =='test':
    df_test=df_test.sample(frac=sample)

Testing the hypothesis

**Test a number of scoring proposals. **

In [ ]:
df=jigsaw_df.copy()

In [ ]:
df.head()

In [ ]:
#df = df.dropna()

In [ ]:
df['score1']=0.16*df['obscene']+0.32*df['toxic']+1.5*df['threat']+0.64*df['insult']+1.5*df['severe_toxic']+1.5*df['identity_hate']
df['score2']=1-jigsaw_df["neutral_speech_agreement"]+1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1*df['identity_hate']
df['score3']=1-jigsaw_df["neutral_speech_agreement"]+1*df['obscene']+1*df['toxic']+1*df['threat']+1*df['insult']+2*df['severe_toxic']+1.5*df['identity_hate']
df['score4']=1*df['obscene']+1*df['toxic']+1*df['insult']+5*df['jigsaw_df_severity']*df['offensive_agreement_rating']+5*df['jigsaw_df_severity']*df['hate_speech_agreement']
df['score5']=1-df["neutral_speech_agreement"]+2*df['cutoff_offensive']+3*df['cutoff_hate']
df['score6']=((df["neutral_speech_agreement"]+0.1)**(df["neutral_speech_agreement"]+0.01))+df['cutoff_offensive']+df['cutoff_hate']
df['score7']=1-jigsaw_df["neutral_speech_agreement"]+0.16*df['obscene']+0.32*df['toxic']+1.5*df['threat']+0.64*df['insult']+1.5*df['severe_toxic']+1.5*df['identity_hate']
df['score8']=1-df["neutral_speech_agreement"]+df['cutoff_offensive']+df["hate_speech_agreement"]
df['score9']=1-df["neutral_speech_agreement"]+df["severity_filter"]*df["offensive_agreement_rating"]+2*df["severity_filter"]*df["hate_speech_agreement"]



In [ ]:
df = df.dropna()

In [ ]:
column_names=['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7', 'score8','score9']

In [ ]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(df, test_size=0.20)
#valid, test = train_test_split(valid, test_size=0.3)

In [ ]:
X_train = train['text'].values
y_train = train.loc[:,column_names].to_numpy()

valid_data = valid['text'].values
valid_target = valid.loc[:,column_names].to_numpy()

print(X_train.shape, y_train.shape)
print(valid_data.shape, valid_target.shape)

in my opinion 0.668 for validation is not bad for an independent model.

In [ ]:
X_less_toxic =df_test.less_toxic.values
X_more_toxic =df_test.more_toxic.values
X_comments_to_score =comments_to_score.text.values


In [ ]:
vectorizor = TfidfVectorizer(analyzer='char_wb', max_df=0.8, min_df=1, ngram_range=(1, 5) )

In [ ]:
#jigsaw_df["proposed_score"]=1-jigsaw_df["neutral_speech_agreement"]+2*jigsaw_df['cutoff_offensive']+3*jigsaw_df['cutoff_hate']

In [ ]:
%%time
X = vectorizor.fit_transform(X_train)
#temp_score = jigsaw_df["proposed_score"].values
y=np.around ( y_train ,decimals = 3)

In [ ]:
print(X.shape,y.shape)

In [ ]:
%%time
regressor=Ridge(alpha=0.5)
regressor.fit(X, y)

In [ ]:
def compare(p1,p2):
    z=0
    k=0
    n=len(p1)
    for i in range(len(p1)):
        if p1[i]< p2[i]:
            z=z+1
        else:
            k=k+1
            #print('nu')
    print(z/n)
    return z/n

In [ ]:
def compare_list(p1,p2):

    for j in range(9):
            z=0
            k=0
            n=len(p1)
            print("score",j+1,"score",j+1,)
            for i in range(len(p1)):
                if p1[i][j]< p2[i][j]:
                    z=z+1
                else:
                    k=k+1
                #print('nu')
            print(z/n)
            #print("p1 score",j,"p2 score",j,z/n)
    return z/n

In [ ]:
#comments_to_score_set=vectorizor.transform(comments_to_score['text'])
#preds = reg.predict(X_less_toxic).squeeze().tolist()
#p1 = predictor.predict(X_less_toxic)
p11 = regressor.predict(vectorizor.transform(X_less_toxic))
#p2 = predictor.predict(X_more_toxic)
p12 = regressor.predict(vectorizor.transform(X_more_toxic))
# Validation Accuracy
compare_list(p11,p12)

In [ ]:
# Get the predictions 
scored_sub=regressor.predict(vectorizor.transform(X_comments_to_score)).squeeze().tolist()

In [ ]:
comments_to_score['scores']=scored_sub

In [ ]:
scores=pd.DataFrame(comments_to_score['scores'].to_list(), columns=['score1','score2', 'score3', 'score4', 'score5', 'score6', 'score7', 'score8', 'score9'])

In [ ]:
colnames=scores.columns

In [ ]:
for col in colnames:
    rank="rank"+col
    comments_to_score[rank]=rankdata(scores[col], method='ordinal')
    #print(rank)

In [ ]:
comments_to_score.head()

In [ ]:
#comments_to_score.columns
cols=['rankscore1', 'rankscore2', 'rankscore3',
       'rankscore4', 'rankscore5', 'rankscore6', 'rankscore7', 'rankscore8',
       'rankscore9']

In [ ]:
comments_to_score['score'] = comments_to_score[cols].sum(axis=1)

In [ ]:
comments_to_score['score'] = comments_to_score['rankscore3']

In [ ]:
#comments_to_score['grand_slam_ranks']=rankdata( comments_to_score['score'], method='ordinal')

In [ ]:
#submission=comments_to_score.loc[:,['grand_slam_ranks']]

In [ ]:
submission=comments_to_score.loc[:,['score']]

In [ ]:
#submission.rename(columns={"grand_slam_ranks": 'score'}, inplace = True)

In [ ]:
submission.to_csv('submission.csv')

In [ ]:
submission.head()